In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Environment setup complete.")

Environment setup complete.


## 1. Load Merged Data

In [2]:
# Define paths
BASE_DIR = Path(r'c:\Users\miray\Desktop\dengue_forecasting_project')
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'

# Load merged data
merged_file = PROCESSED_DIR / 'merged_training_data.csv'
df = pd.read_csv(merged_file, parse_dates=['Date'], index_col='Date')

print(f"Loaded merged data: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Columns: {len(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

Loaded merged data: (326, 52)
Date range: 2016-01-08 00:00:00 to 2022-10-01 00:00:00
Columns: 52

First few rows:
            Aklan_Humidity  Aklan_Rainfall  Aklan_Temp_Avg  Aklan_Temp_Max  \
Date                                                                         
2016-01-08           81.81            2.32           26.61           30.03   
2016-01-15           77.71            4.20           26.07           29.97   
2016-01-22           82.18           53.54           25.59           28.77   
2016-01-29           84.27           12.23           26.19           29.47   
2016-02-05           82.06           22.26           26.06           29.64   

            Aklan_Temp_Min  Antique_Humidity  Antique_Rainfall  \
Date                                                             
2016-01-08           24.24             77.28              1.38   
2016-01-15           23.47             73.18              4.33   
2016-01-22           23.40             77.30             32.10   
2016-01-2

## 2. Identify Feature Groups

In [3]:
# Categorize columns
weather_cols = [col for col in df.columns if any(x in col for x in ['Humidity', 'Rainfall', 'Temp'])]
trends_cols = [col for col in df.columns if 'Search' in col]
dengue_cols = [col for col in df.columns if 'Cases' in col]

print(f"Feature groups:")
print(f"  Weather: {len(weather_cols)} columns")
print(f"  Search trends: {len(trends_cols)} columns")
print(f"  Dengue cases (targets): {len(dengue_cols)} columns")

print(f"\nDengue target columns:")
print(dengue_cols)

Feature groups:
  Weather: 40 columns
  Search trends: 4 columns
  Dengue cases (targets): 8 columns

Dengue target columns:
['Aklan_Cases', 'Antique_Cases', 'Bacolodcity_Cases', 'Capiz_Cases', 'Guimaras_Cases', 'Iloilo_Cases', 'Iloilocity_Cases', 'Negrosocc_Cases']


## 3. Create Lag Features (t-1 to t-4 weeks)

In [4]:
# Create lag features for weather and dengue
# Lags: 1, 2, 3, 4 weeks back
lag_weeks = [1, 2, 3, 4]

print("Creating lag features...")
df_lagged = df.copy()

# Lag weather features (all 40 columns)
for col in weather_cols:
    for lag in lag_weeks:
        df_lagged[f"{col}_lag{lag}"] = df[col].shift(lag)

print(f"  ✅ Created {len(weather_cols) * len(lag_weeks)} weather lag features")

# Lag dengue cases (past cases help predict future cases)
for col in dengue_cols:
    for lag in lag_weeks:
        df_lagged[f"{col}_lag{lag}"] = df[col].shift(lag)

print(f"  ✅ Created {len(dengue_cols) * len(lag_weeks)} dengue lag features")

# Lag search trends
for col in trends_cols:
    for lag in lag_weeks:
        df_lagged[f"{col}_lag{lag}"] = df[col].shift(lag)

print(f"  ✅ Created {len(trends_cols) * len(lag_weeks)} search trends lag features")

print(f"\nTotal columns after lagging: {len(df_lagged.columns)}")

Creating lag features...
  ✅ Created 160 weather lag features
  ✅ Created 32 dengue lag features
  ✅ Created 16 search trends lag features

Total columns after lagging: 260


## 4. Create Rolling Statistics (3-week and 4-week means)

In [5]:
# Create rolling means for dengue cases (trend indicators)
print("Creating rolling statistics...")

for col in dengue_cols:
    # 3-week rolling mean
    df_lagged[f"{col}_roll3"] = df[col].shift(1).rolling(window=3, min_periods=1).mean()
    # 4-week rolling mean
    df_lagged[f"{col}_roll4"] = df[col].shift(1).rolling(window=4, min_periods=1).mean()

print(f"  ✅ Created {len(dengue_cols) * 2} rolling statistics for dengue cases")

# Rolling means for key weather variables (rainfall and temperature)
key_weather = [col for col in weather_cols if 'Rainfall' in col or 'Temp_Avg' in col]

for col in key_weather:
    df_lagged[f"{col}_roll3"] = df[col].shift(1).rolling(window=3, min_periods=1).mean()
    df_lagged[f"{col}_roll4"] = df[col].shift(1).rolling(window=4, min_periods=1).mean()

print(f"  ✅ Created {len(key_weather) * 2} rolling statistics for key weather variables")

print(f"\nTotal columns after rolling stats: {len(df_lagged.columns)}")

Creating rolling statistics...
  ✅ Created 16 rolling statistics for dengue cases
  ✅ Created 32 rolling statistics for key weather variables

Total columns after rolling stats: 308


## 5. Create Temporal Features

In [6]:
# Add temporal features
print("Creating temporal features...")

df_lagged['month'] = df_lagged.index.month
df_lagged['week_of_year'] = df_lagged.index.isocalendar().week
df_lagged['quarter'] = df_lagged.index.quarter

# Cyclical encoding for month and week (sine/cosine transformation)
df_lagged['month_sin'] = np.sin(2 * np.pi * df_lagged['month'] / 12)
df_lagged['month_cos'] = np.cos(2 * np.pi * df_lagged['month'] / 12)
df_lagged['week_sin'] = np.sin(2 * np.pi * df_lagged['week_of_year'] / 52)
df_lagged['week_cos'] = np.cos(2 * np.pi * df_lagged['week_of_year'] / 52)

# Season indicators (Philippines: Dry season Nov-Apr, Wet season May-Oct)
df_lagged['is_wet_season'] = df_lagged['month'].isin([5, 6, 7, 8, 9, 10]).astype(int)
df_lagged['is_dry_season'] = df_lagged['month'].isin([11, 12, 1, 2, 3, 4]).astype(int)

print(f"  ✅ Created 9 temporal features")
print(f"\nTemporal features: month, week_of_year, quarter, month_sin, month_cos, week_sin, week_cos, is_wet_season, is_dry_season")
print(f"\nTotal columns: {len(df_lagged.columns)}")

Creating temporal features...
  ✅ Created 9 temporal features

Temporal features: month, week_of_year, quarter, month_sin, month_cos, week_sin, week_cos, is_wet_season, is_dry_season

Total columns: 317


## 6. Create Target Variables (t+4 weeks ahead)

In [7]:
# Create target: dengue cases 4 weeks into the future
print("Creating target variables (t+4 weeks ahead)...")

target_cols = []
for col in dengue_cols:
    target_name = f"{col}_target"
    df_lagged[target_name] = df[col].shift(-4)  # Shift backward to get future values
    target_cols.append(target_name)

print(f"  ✅ Created {len(target_cols)} target columns (4 weeks ahead)")
print(f"\nTarget columns:")
print(target_cols)

print(f"\nTotal columns: {len(df_lagged.columns)}")

Creating target variables (t+4 weeks ahead)...
  ✅ Created 8 target columns (4 weeks ahead)

Target columns:
['Aklan_Cases_target', 'Antique_Cases_target', 'Bacolodcity_Cases_target', 'Capiz_Cases_target', 'Guimaras_Cases_target', 'Iloilo_Cases_target', 'Iloilocity_Cases_target', 'Negrosocc_Cases_target']

Total columns: 325


## 7. Remove Rows with Missing Values (due to lag/lead operations)

In [8]:
print(f"\nBefore removing NaN rows: {len(df_lagged)}")
print(f"Missing values: {df_lagged.isnull().sum().sum()}")

# Drop rows with any NaN (first 4 rows due to lags, last 4 rows due to targets)
df_final = df_lagged.dropna()

print(f"\nAfter removing NaN rows: {len(df_final)}")
print(f"Date range: {df_final.index.min()} to {df_final.index.max()}")
print(f"Missing values: {df_final.isnull().sum().sum()}")
print(f"\nRows removed: {len(df_lagged) - len(df_final)} (first 4 for lags, last 4 for targets)")


Before removing NaN rows: 326
Missing values: 600

After removing NaN rows: 318
Date range: 2016-02-05 00:00:00 to 2022-09-03 00:00:00
Missing values: 0

Rows removed: 8 (first 4 for lags, last 4 for targets)


## 8. Organize Columns (Features → Targets)

In [9]:
# Separate features and targets
feature_cols = [col for col in df_final.columns if '_target' not in col]
target_cols = [col for col in df_final.columns if '_target' in col]

# Reorder: all features first, then all targets
df_final = df_final[feature_cols + target_cols]

print(f"Column organization:")
print(f"  Feature columns: {len(feature_cols)}")
print(f"  Target columns: {len(target_cols)}")
print(f"  Total: {len(df_final.columns)}")

print(f"\nFeature breakdown:")
print(f"  Original weather: {len(weather_cols)}")
print(f"  Original trends: {len(trends_cols)}")
print(f"  Original dengue: {len(dengue_cols)}")
print(f"  Lag features: {(len(weather_cols) + len(dengue_cols) + len(trends_cols)) * len(lag_weeks)}")
print(f"  Rolling statistics: {(len(dengue_cols) + len(key_weather)) * 2}")
print(f"  Temporal features: 9")

Column organization:
  Feature columns: 317
  Target columns: 8
  Total: 325

Feature breakdown:
  Original weather: 40
  Original trends: 4
  Original dengue: 8
  Lag features: 208
  Rolling statistics: 48
  Temporal features: 9


## 9. Summary Statistics

In [10]:
print("="*70)
print("FEATURE ENGINEERING SUMMARY")
print("="*70)

print(f"\n📊 Final Dataset Shape: {df_final.shape}")
print(f"   Samples (weeks): {len(df_final)}")
print(f"   Features: {len(feature_cols)}")
print(f"   Targets: {len(target_cols)}")

print(f"\n📅 Date Range:")
print(f"   Start: {df_final.index.min().strftime('%Y-%m-%d')}")
print(f"   End: {df_final.index.max().strftime('%Y-%m-%d')}")

print(f"\n📈 Target Statistics (4 weeks ahead):")
print(df_final[target_cols].describe().loc[['count', 'mean', 'std', 'min', 'max']])

print(f"\n✅ Data Quality:")
print(f"   Missing values: {df_final.isnull().sum().sum()}")
print(f"   Infinite values: {np.isinf(df_final.select_dtypes(include=[np.number])).sum().sum()}")

# Check for any remaining issues
if df_final.isnull().sum().sum() > 0:
    print("\n⚠️ Columns with missing values:")
    print(df_final.isnull().sum()[df_final.isnull().sum() > 0])

FEATURE ENGINEERING SUMMARY

📊 Final Dataset Shape: (318, 325)
   Samples (weeks): 318
   Features: 317
   Targets: 8

📅 Date Range:
   Start: 2016-02-05
   End: 2022-09-03

📈 Target Statistics (4 weeks ahead):
       Aklan_Cases_target  Antique_Cases_target  Bacolodcity_Cases_target  \
count          318.000000            318.000000                318.000000   
mean            34.993711             37.066038                 27.371069   
std             63.876063            125.520187                 36.117588   
min              0.000000              0.000000                  0.000000   
max            473.000000           2036.000000                187.000000   

       Capiz_Cases_target  Guimaras_Cases_target  Iloilo_Cases_target  \
count          318.000000             318.000000           318.000000   
mean            35.349057              10.761006           121.704403   
std             75.036653              31.219576           306.938840   
min              0.000000         

## 10. Save Engineered Dataset

In [12]:
# Save engineered features
output_file = PROCESSED_DIR / 'feature_engineered_data.csv'
df_final.to_csv(output_file, date_format='%Y-%m-%d')

print(f"\n✅ SAVED: {output_file}")
print(f"   Shape: {df_final.shape}")
print(f"   Size: {output_file.stat().st_size / 1024:.1f} KB")

# Also save a feature list for reference
feature_list_file = PROCESSED_DIR / 'feature_list.txt'
with open(feature_list_file, 'w') as f:
    f.write("FEATURE COLUMNS\n")
    f.write("="*50 + "\n")
    for i, col in enumerate(feature_cols, 1):
        f.write(f"{i}. {col}\n")
    f.write("\n" + "="*50 + "\n")
    f.write("TARGET COLUMNS\n")
    f.write("="*50 + "\n")
    for i, col in enumerate(target_cols, 1):
        f.write(f"{i}. {col}\n")

print(f"\n✅ SAVED: {feature_list_file}")


✅ SAVED: c:\Users\miray\Desktop\dengue_forecasting_project\data\processed\feature_engineered_data.csv
   Shape: (318, 325)
   Size: 746.8 KB

✅ SAVED: c:\Users\miray\Desktop\dengue_forecasting_project\data\processed\feature_list.txt
